In [1]:
!pip install openai pandas > /dev/null

In [2]:
import openai
import re
import pandas as pd
import os
import json

from IPython.display import display

In [3]:
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('API_KEY')

In [4]:
json_data = ''
with open('/content/2023_11_KICE.json', 'r', encoding='utf-8') as f:
  json_data = json.loads(f.read())

In [15]:
# 결과 저장을 위한 변수들 초기화
all_answers = []
total_score = 0
max_possible_score = 0  # 문제별 점수를 합산하여 계산
incorrect_questions = []

In [16]:
# 지시어와 의미 통합을 강화한 문제 풀이 함수
def solve_with_gpt4o_enhanced(paragraph, question, question_plus, choices):
    try:
        # 1. 지시어 분석 및 보기 전처리
        analysis_prompt = f"""
다음 문단을 분석하여 지시어(이, 그, 저, 이것, 그것, 이러한, 그러한 등)와 그것이 가리키는 대상을 식별해주세요.
지시어와 대상을 표 형태로 정리해주세요.

문단: {paragraph}

형식:
지시어 | 해당 지시어가 가리키는 대상
"""

        # 지시어 분석 요청
        analysis_response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": analysis_prompt}],
            temperature=0,
            max_tokens=500
        )

        reference_analysis = analysis_response.choices[0].message.content.strip()

        # 2. 분석 결과를 활용한 보기 전처리
        preprocessing_prompt = f"""
다음은 지문과 그 안의 지시어 분석 결과입니다:

지문: {paragraph}

지시어 분석:
{reference_analysis}

아래의 보기들을 검토해서, 지시어가 포함된 경우 지시어가 가리키는 정확한 대상으로 대체하여 수정된 보기를 제공해주세요.
각 보기마다 원본과 수정된 버전을 제시해주세요.

보기:
{chr(10).join([f"{i+1}. {choice}" for i, choice in enumerate(choices)])}

형식:
보기 번호: 원본 -> 수정된 버전 (수정이 필요 없으면 "수정 필요 없음")
"""

        # 보기 전처리 요청
        preprocessing_response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": preprocessing_prompt}],
            temperature=0,
            max_tokens=1000
        )

        processed_choices = preprocessing_response.choices[0].message.content.strip()

        # 3. 강화된 의미 통합으로 문제 풀이
        enhanced_prompt = f"""다음 지문과 질문에 가장 적절한 답을 골라주세요.

지문: {paragraph}

질문: {question}
{f"추가 설명: {question_plus}" if question_plus else ""}

원본 보기:
{chr(10).join([f"{i+1}. {choice}" for i, choice in enumerate(choices)])}

지시어 분석 및 보기 전처리 결과:
{processed_choices}

### 분석 지침:
1. 질문이 요구하는 것을 정확히 파악하세요.
2. 의미 통합 과정:
   - 지문의 여러 부분에서 관련 정보를 추출하고 통합하세요.
   - 명시적으로 언급되지 않았지만 여러 문장을 종합하면 도출되는 결론을 찾으세요.
   - 지문의 전체 맥락을 고려하여 일관된 해석을 도출하세요.
3. 각 보기를 평가할 때:
   - 지시어가 해석된 보기의 내용이 지문과 일치하는지 검토하세요.
   - 명시적 진술뿐 아니라 함의된 내용까지 고려하세요.
   - 지문의 여러 부분을 연결해 얻을 수 있는 통찰인지 확인하세요.

먼저 전체적인 의미 통합 분석을 수행한 후, 각 보기를 하나씩 검토하세요.

답변 형식: "따라서 정답은 X번입니다."로 결론을 명확히 제시해주세요.
"""

        # 최종 문제 풀이 요청
        solving_response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": enhanced_prompt}],
            temperature=0,
            max_tokens=1500
        )

        full_response = solving_response.choices[0].message.content.strip()
        print(f'질문: {question}\n')
        print(f'지시어 분석:\n{reference_analysis}\n')
        print(f'보기 전처리:\n{processed_choices}\n')
        print(f'LLM 분석:\n{full_response}\n')

        # 응답에서 선택한 답변 번호 추출
        match = re.search(r'정답은\s*(\d+)', full_response)
        if not match:
            match = re.search(r'(\d+)[번호\.]\s*[이가]?\s*정답', full_response)
        if not match:
            match = re.search(r'(\d+)[번호\.]\s*선택', full_response)
        if not match:
            match = re.search(r'(\d+)', full_response)

        if match:
            answer_number = int(match.group(1))
            # 완전한 분석 정보를 포함하여 반환
            complete_analysis = f"지시어 분석:\n{reference_analysis}\n\n보기 전처리:\n{processed_choices}\n\n최종 분석:\n{full_response}"
            return answer_number, complete_analysis
        else:
            print(f"답변 형식이 맞지 않습니다. 전체 답변: {full_response}")
            return None, full_response

    except Exception as e:
        print(f"GPT-4o 호출 오류: {e}")
        return None, f"오류 발생: {e}"

In [17]:
global_problem_number = 0

# 모든 문제 풀기
def process_data(json_data):
    global total_score, max_possible_score, global_problem_number

    for data_set in json_data:
        if not data_set:  # 빈 객체인 경우 스킵
            continue

        paragraph_id = data_set.get("id", "Unknown ID")
        paragraph = data_set.get("paragraph", "")
        problems = data_set.get("problems", [])

        print(f"\n지문 ID: {paragraph_id} 문제 풀이 시작")

        for problem in problems:
            global_problem_number += 1  # 전체 문제 번호 증가

            question = problem.get("question", "")
            question_plus = problem.get("question_plus", "")
            choices = problem.get("choices", [])
            correct_answer = problem.get("answer", 0)
            problem_score = problem.get("score", 0)

            # 최대 가능 점수 업데이트
            if isinstance(problem_score, (int, float)) and problem_score >= 0:
                max_possible_score += problem_score
            else:
                print(f"⚠️ 문제 {global_problem_number}에 유효하지 않은 점수: {problem_score} (0으로 처리됨)")

            # GPT-4o로 문제 풀이
            predicted_answer, reasoning = solve_with_gpt4o_enhanced(paragraph, question, question_plus, choices)

            if predicted_answer is None:
                print(f"전체 문제 {global_problem_number}: GPT-4o가 답변을 생성하지 못했습니다. 기본값 0을 사용합니다.")
                predicted_answer = 0

            # 채점
            is_correct = (predicted_answer == correct_answer)
            earned_score = problem_score if is_correct else 0
            total_score += earned_score

            # 오답인 경우에만 출력
            if not is_correct:
                print(f"전체 문제 {global_problem_number} 결과: 오답 (선택: {predicted_answer}, 정답: {correct_answer})")

            # 결과 저장
            result = {
                'paragraph_id': paragraph_id,
                'problem_number': global_problem_number,
                'question': question[:50] + '...' if len(question) > 50 else question,
                'predicted_answer': predicted_answer,
                'correct_answer': correct_answer,
                'is_correct': is_correct,
                'score': earned_score,
                'possible_score': problem_score,
                'reasoning': reasoning
            }
            all_answers.append(result)

            # 틀린 문제 저장
            if not is_correct:
                result_full = result.copy()
                result_full['question'] = question
                result_full['question_plus'] = question_plus
                result_full['choices'] = choices
                incorrect_questions.append(result_full)


In [18]:
# 주어진 JSON 데이터에서 문제 추출 및 처리
def analyze_json_data(json_data):
    # 데이터 처리
    process_data(json_data)

    # 틀린 문제 목록 출력
    if incorrect_questions:
        print("\n===== 틀린 문제 목록 =====")
        for q in incorrect_questions:
            print(f"문제 {q['paragraph_id']} - {q['problem_number']}")
            print(f"질문: {q['question']}")
            if q.get('question_plus'):
                print(f"추가 설명: {q['question_plus']}")
            print(f"GPT-4o 선택: {q['predicted_answer']}, 정답: {q['correct_answer']}")
            print("보기:")
            for j, choice in enumerate(q['choices']):
                print(f"{j+1}. {choice}")
            print(f"사고 과정:\n{q['reasoning']}\n")
    else:
        print("\n모든 문제를 맞추셨습니다!")

    # 틀린 문제만 필터링해서 DataFrame으로 변환
    df_incorrect = pd.DataFrame([
        {k: v for k, v in item.items() if k != 'reasoning'}
        for item in all_answers if not item['is_correct']
    ])

    print("\n===== 틀린 문제별 답안 및 점수 =====")

    if not df_incorrect.empty:
        display(df_incorrect)
    else:
        print("🎉 모든 문제를 맞추셨습니다!")

    # 결과 출력
    print("\n===== 채점 결과 =====")
    print(f"총점: {total_score} / {max_possible_score}")

## 1번째 시도
- 총점: 87 / 100
- 틀린문제: [2, 10, 13, 15, 33, 42]
  - 원인 (개인적인 추측)
    - 2번
      - '그' 와 같은 지시 대명사를 제대로 이해x
        - 지시 대명사가 의미 하는바를 붙여주기 ex> 그는 → 그 베토벤 는
        - Coreference Resolution (지시어 해소)
          - 실제 지칭하는 명사로 대체
            - 베토벤은 멋있다. 그는 남자다. → 베토벤은 멋있다. 베토벤은 남자다.
    - 10번
      - 법령의 조문은 ~ <u>요건과 효과로 구성</u>된 ~ / 법조문에는 구체적 상황을 <u>불확정 개념이 사용</u>될 수 있기 ~  => 따라서 요건과 효과는 불확정 개념 사용 가능  
        - 의미 통합(Semantic Integration) 또는 함의 관계에 기반한 연역적 추론
    - 13번
      -
    - 15번
      -
    - 33번
      -
    - 42번
      - (나), 지도(시각정보)를 모두 활용 못함
        - pass

In [13]:
analyze_json_data(json_data)


지문 ID: 2023_11_KICE_1-3 문제 풀이 시작
문제 1 풀이 중...
질문: 윗글의 내용과 일치하지 않는 것은?

LLM 분석:
먼저 각 보기를 분석하고 지문의 내용과 비교하여 정답을 도출하겠습니다.

1. "같은 책을 읽은 독자라도 서로 다른 의미를 구성할 수 있다."
   - 지문에서는 "같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다."라고 설명하고 있습니다. 따라서 1번은 지문과 일치합니다.

2. "다른 독자와의 소통은 독자가 인식의 폭을 확장하도록 돕는다."
   - 지문에서는 "비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는 자신의 인식을 심화 확장할 수 있다."라고 설명하고 있습니다. 따라서 2번은 지문과 일치합니다.

3. "독자는 직접 경험해 보지 못했던 다양한 삶을 책의 필자를 매개로 접할 수 있다."
   - 지문에서는 "직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를 바라볼 수 있다."라고 설명하고 있습니다. 따라서 3번은 지문과 일치합니다.

4. "독자의 배경지식, 관점, 읽기 환경, 과제는 독자의 의미 구성에 영향을 주는 독자 요인이다."
   - 지문에서는 "독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인"이라고 설명하고 있습니다. 여기서 '읽기 환경'과 '과제'는 '상황 요인'으로 분류되며, '독자 요인'이 아닙니다. 따라서 4번은 지문과 일치하지 않습니다.

5. "독자는 책을 읽을 때 자신이 속한 사회나 시대의 영향을 받으며 필자와 간접적으로 대화한다."
   - 지문에서는 "독자는 자신이 속한 사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 독서는 필자와 간접적으로 대

,paragraph_id,problem_number,question,predicted_answer,correct_answer,is_correct,score,possible_score
0,2023_11_KICE_1-3,1,윗글의 내용과 일치하지 않는 것은?,4,4,True,2,2
1,2023_11_KICE_1-3,2,다음은 학생이 독서 후 작성한 글의 일부이다. [A]를 바탕으로 ⓐ～ⓔ를 이해한 내...,3,5,False,0,3
2,2023_11_KICE_1-3,3,윗글을 읽고 ㉠에 대해 보인 반응으로 적절하지 않은 것은?,1,1,True,2,2
3,2023_11_KICE_4-9,1,(가)와 (나)에 대한 설명으로 가장 적절한 것은?,4,4,True,2,2
4,2023_11_KICE_4-9,2,[A]에 대한 이해로 적절하지 않은 것은?,5,5,True,2,2
5,2023_11_KICE_4-9,3,"㉮에 대한 이해를 바탕으로 ㉠, ㉡에 대해 파악한 내용으로 적절하지 않은 것은?",3,3,True,2,2
6,2023_11_KICE_4-9,4,㉯를 반박하기 위한 ‘이수광’의 말로 가장 적절한 것은?,2,2,True,2,2
7,2023_11_KICE_4-9,5,"(가), (나)를 읽은 학생이 <보기>의 임원경제지에 대해 보인 반응으로 적절하지 ...",5,5,True,3,3
8,2023_11_KICE_4-9,6,문맥상 ⓐ～ⓔ와 바꾸어 쓰기에 적절하지 않은 것은?,2,2,True,2,2
9,2023_11_KICE_10-13,1,윗글의 내용과 일치하지 않는 것은?,1,4,False,0,2



정답 개수: 39/45
정답률: 86.67%


## 2번째 시도
- 총점: 89 / 100
- 틀린문제: [2, 10, 13, 15, 42]

In [9]:
analyze_json_data(json_data)


지문 ID: 2023_11_KICE_1-3 문제 풀이 시작
전체 문제 1 풀이 중...
질문: 윗글의 내용과 일치하지 않는 것은?

LLM 분석:
질문은 "윗글의 내용과 일치하지 않는 것은?"입니다. 즉, 주어진 보기 중에서 지문의 내용과 맞지 않는 것을 찾는 것이 목표입니다.

지문을 분석해보면, 독서의 즐거움, 특히 소통의 즐거움에 대해 설명하고 있습니다. 독자는 책과의 소통을 통해 필자와 간접적으로 대화하며, 다양한 삶을 경험하고, 다른 독자와의 소통을 통해 인식을 확장할 수 있다고 합니다. 또한, 독자의 배경지식, 관점, 읽기 환경, 과제 등이 독자의 의미 구성에 영향을 미친다고 설명합니다.

이제 각 보기를 지문과 비교하여 분석해보겠습니다.

1. **같은 책을 읽은 독자라도 서로 다른 의미를 구성할 수 있다.**
   - 지문에서 "같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다."라고 명시하고 있습니다. 따라서 이 내용은 지문과 일치합니다.

2. **다른 독자와의 소통은 독자가 인식의 폭을 확장하도록 돕는다.**
   - 지문에서 "비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는 자신의 인식을 심화 확장할 수 있다."라고 설명하고 있습니다. 따라서 이 내용도 지문과 일치합니다.

3. **독자는 직접 경험해 보지 못했던 다양한 삶을 책의 필자를 매개로 접할 수 있다.**
   - 지문에서 "직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를 바라볼 수 있다."라고 설명하고 있습니다. 따라서 이 내용도 지문과 일치합니다.

4. **독자의 배경지식, 관점, 읽기 환경, 과제는 독자의 의미 구성에 영향을 주는 독자 요인이다.**
   - 지문에서는 "독자의 배경지식이나 관점 등

,paragraph_id,problem_number,question,predicted_answer,correct_answer,is_correct,score,possible_score
0,2023_11_KICE_1-3,2,다음은 학생이 독서 후 작성한 글의 일부이다. [A]를 바탕으로 ⓐ～ⓔ를 이해한 내...,4,5,False,0,3
1,2023_11_KICE_10-13,10,윗글의 내용과 일치하지 않는 것은?,1,4,False,0,2
2,2023_11_KICE_10-13,13,문맥상 ⓐ～ⓔ의 의미와 가장 가까운 것은?,1,5,False,0,2
3,2023_11_KICE_14-17,15,윗글을 읽고 추론한 내용으로 적절하지 않은 것은?,5,4,False,0,2
4,2023_11_KICE_38-42,42,다음은 (나)를 바탕으로 학생들이 만든 지도의 초안이다. ㉠～㉤에 대한 반응으로 가...,3,4,False,0,2


## 3번째 시도
- 총점: 89 / 100
- 틀린문제: [2, 13, 15, 33, 42]

In [14]:
analyze_json_data(json_data)


지문 ID: 2023_11_KICE_1-3 문제 풀이 시작
질문: 윗글의 내용과 일치하지 않는 것은?

LLM 분석:
### 분석 단계

1. **질문 분석:**
   - 질문은 "윗글의 내용과 일치하지 않는 것은?"입니다. 즉, 지문과 불일치하는 내용을 찾는 것이 목표입니다.

2. **지문 분석:**
   - 지문은 독서의 즐거움, 특히 소통의 즐거움에 대해 설명하고 있습니다.
   - 독자는 책과 소통하며, 필자와 간접적으로 대화하고, 다양한 삶을 경험할 수 있습니다.
   - 같은 책을 읽어도 독자의 배경지식, 관점, 읽기 환경, 과제 등의 요인에 따라 서로 다른 의미를 구성할 수 있습니다.
   - 독자는 다른 독자와 소통하며 인식의 폭을 확장할 수 있습니다.
   - 온라인 소통 공간의 확대로 독서 경험을 공유하고, 책을 읽지 않은 사람들에게도 책과 소통하도록 돕는 것이 가능합니다.

3. **보기 분석:**
   - **1번 보기:** "같은 책을 읽은 독자라도 서로 다른 의미를 구성할 수 있다."
     - 지문에서 "같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다."라고 명시되어 있습니다. 따라서 지문과 일치합니다.
   
   - **2번 보기:** "다른 독자와의 소통은 독자가 인식의 폭을 확장하도록 돕는다."
     - 지문에서 "비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는 자신의 인식을 심화 확장할 수 있다."라고 설명하고 있습니다. 따라서 지문과 일치합니다.
   
   - **3번 보기:** "독자는 직접 경험해 보지 못했던 다양한 삶을 책의 필자를 매개로 접할 수 있다."
     - 지문에서 "직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를 바라볼

,paragraph_id,problem_number,question,predicted_answer,correct_answer,is_correct,score,possible_score
0,2023_11_KICE_1-3,2,다음은 학생이 독서 후 작성한 글의 일부이다. [A]를 바탕으로 ⓐ～ⓔ를 이해한 내...,3,5,False,0,3
1,2023_11_KICE_10-13,13,문맥상 ⓐ～ⓔ의 의미와 가장 가까운 것은?,1,5,False,0,2
2,2023_11_KICE_14-17,15,윗글을 읽고 추론한 내용으로 적절하지 않은 것은?,5,4,False,0,2
3,2023_11_KICE_31-34,33,[A]～[F]에 대한 이해로 가장 적절한 것은?,3,2,False,0,2
4,2023_11_KICE_38-42,42,다음은 (나)를 바탕으로 학생들이 만든 지도의 초안이다. ㉠～㉤에 대한 반응으로 가...,2,4,False,0,2



===== 채점 결과 =====
총점: 89 / 100


## 4번째 시도
- 총점: 82 / 100
- 틀린문제: [2,10,13,15,22,38,41,42]
  - 기존에 맞던 문제에서 지시어를 잘못 유추한 경우 오히려 답을 틀림

In [19]:
analyze_json_data(json_data)


지문 ID: 2023_11_KICE_1-3 문제 풀이 시작
질문: 윗글의 내용과 일치하지 않는 것은?

지시어 분석:
지시어 | 해당 지시어가 가리키는 대상  
---|---  
그 | 독서의 즐거움의 중심  
이때 | 같은 책을 읽은 독자들이 필자가 보여주는 세계를 수용하지 않고 저마다 다른 의미를 구성하는 시점  
저마다 | 각 독자  
이러한 | 소통의 과정  
전자의 | 책에서 답을 찾는 질문  
후자의 | 독자 자신에게서 답을 찾는 질문  
이런 | 질문들  
이에 | 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일  
자신의 | 독자 본인  
그것 | 책을 읽지 않은 타인이 책과 소통하도록 돕는 일

보기 전처리:
1. 같은 책을 읽은 독자라도 서로 다른 의미를 구성할 수 있다. -> 같은 책을 읽은 독자라도 각 독자는 서로 다른 의미를 구성할 수 있다.
2. 다른 독자와의 소통은 독자가 인식의 폭을 확장하도록 돕는다 -> 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일은 독자가 인식의 폭을 확장하도록 돕는다.
3. 독자는 직접 경험해 보지 못했던 다양한 삶을 책의 필자를 매개로 접할 수 있다. -> 수정 필요 없음
4. 독자의 배경지식, 관점, 읽기 환경, 과제는 독자의 의미 구성에 영향을 주는 독자 요인이다. -> 수정 필요 없음
5. 독자는 책을 읽을 때 자신이 속한 사회나 시대의 영향을 받으며 필자와 간접적으로 대화한다 -> 수정 필요 없음

LLM 분석:
윗글의 내용과 일치하지 않는 것을 찾기 위해 각 보기를 검토해 보겠습니다.

1. "같은 책을 읽은 독자라도 서로 다른 의미를 구성할 수 있다." - 지문에서는 독자의 배경지식이나 관점 등의 요인에 따라 같은 책을 읽어도 서로 다른 의미를 구성할 수 있다고 설명하고 있습니다. 따라서 이 내용은 지문과 일치합니다.

2. "다른 독자와의 소통은 독자가 인식의 폭을 확장하도록 돕는다

,paragraph_id,problem_number,question,predicted_answer,correct_answer,is_correct,score,possible_score
0,2023_11_KICE_1-3,2,다음은 학생이 독서 후 작성한 글의 일부이다. [A]를 바탕으로 ⓐ～ⓔ를 이해한 내...,3,5,False,0,3
1,2023_11_KICE_10-13,10,윗글의 내용과 일치하지 않는 것은?,1,4,False,0,2
2,2023_11_KICE_10-13,13,문맥상 ⓐ～ⓔ의 의미와 가장 가까운 것은?,1,5,False,0,2
3,2023_11_KICE_14-17,15,윗글을 읽고 추론한 내용으로 적절하지 않은 것은?,5,4,False,0,2
4,2023_11_KICE_22-26,22,(가)～(다)의 공통점으로 가장 적절한 것은?,3,1,False,0,2
5,2023_11_KICE_38-42,38,(가)에 활용된 글쓰기 방식으로 가장 적절한 것은?,1,4,False,0,2
6,2023_11_KICE_38-42,41,"(가)와 (나)를 고려할 때, ‘학생 1’이 쓴 [회의록]의 내용 중 적절하지 않은...",1,2,False,0,3
7,2023_11_KICE_38-42,42,다음은 (나)를 바탕으로 학생들이 만든 지도의 초안이다. ㉠～㉤에 대한 반응으로 가...,1,4,False,0,2



===== 채점 결과 =====
총점: 82 / 100
